# 협업 필터링 추천 시스템
 - 취향을 기준으로 군집 나눔

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from utility import *
from sklearn.model_selection import train_test_split

## 3.1 협업 필터링의 원리 
 - 취향이 비슷한 집단이 존재한다는 가정으로 시작함
 - 방법
  1. user와 movie로 값은 rating 점수로 구성된 sparse matrix에서 user 별 correlation 구함
  2. correlation이 높은 user가 공통적으로 좋아하는 영화를 찾음
  3. 공통적으로 좋아하는 영화 중 기존 user가 보지 않은 영화 추천

## 3.2 유사도지표
 1. 상관계수
  - 평가지표가 연속값인 경우 사용
  - -1~1사이의 값을 가지고 -1이면 정반대 1이면 완전 일치를 나타냄
 2. 코사인 유사도
  - 각 아이템을 하나의 차원으로 보고 사용자의 평가값을 좌표값으로 확인
  -  두 사용자 간의 벡터 각도로 유사도 확인
 3. 타니모토 계수
  - 데이터가 이진값을 가질 경우 사용
  - $simil(x,y) = {c \over a+b-c}$
  - a : 사용자 $x$가 1의 값을 갖는(구입 혹은 클릭) 아이템의 수
  - b : 사용자 $y$가 1의 값을 갖는 아이템의 수
  - c : 사용자 $x$와 $y$가 공통적으로 1의 값을 갖는 아이템의 수

## 3.3 기본 CF 알고리즘
 1. 모든 사용자 간 평가 유사도 계산
 2. 현재 추천 대상이 되는 사람과 다른 사용자의 유사도 추출
 3. 현재 사용자가 사용하지 않은 아이템에 대한 예상 평가값 계산
  - 예상 평가값 = 다른 사용자의 평가를 유사도와 함께 가중해서 평균 냄
 4. 가장 높은 N개의 아이템 추천

In [2]:
# 데이터 불러오기
users, movies, ratings = getData()

ratings.drop('timestamp', axis=1, inplace=True)
ratings.reset_index(inplace=True)

x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size = 0.25)

rating_matrix = x_train.pivot(index = 'user_id',columns = 'movie_id',values = 'rating')

In [3]:
# user끼리의 코사인 유사도 계산
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [4]:
# 특정 user와 특정 movie에 대해 가중평균 rating 계산 
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)
        mean_rating = np.dot(movie_ratings,sim_scores)/sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

In [5]:
score(CF_simple,x_test)

1.0161445472708168

## 3.4 이웃을 이용한 CF
 - 유사도가 높은 사람들만을 사용해서 예측 진행
 - 높은 사람 정하는 기준
  1. KNN : 가장 유사한 K명 선택 -> 보통 많이 선택하는 방법
  2. thresholding : 기준을 정해놓고 기준에 충족시키는 사용자 이웃으로 정함 -> 한명도 없을 수도 있음

In [5]:
# neighbor size를 통해 예측치 계산
## neighbor_size = 0인 경우, 전체 사용자 데이터 사용
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        if neighbor_size == 0:
            mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
        else:
            if len(sim_scores)>1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                mean_rating = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

In [14]:
score_neighbor(cf_knn,x_test,neighbor_size=30)

1.003183603043405

In [6]:
# 실제 추천 코드
rating_matrix = ratings.pivot(values='rating',index='user_id',columns='movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size = 30):
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0 
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

In [7]:
recom_movie(user_id=2, n_items=5)

movie_id
1293                         Star Kid (1997)
1189                      Prefontaine (1997)
1467    Saint of Fort Washington, The (1993)
1500               Santa with Muscles (1996)
318                  Schindler's List (1993)
Name: title, dtype: object

## 3.5 최적의 이웃 크기 결정
 - 추천의 정확도를 최대로 하는 이웃크기를 결정할 필요 존재
 - 이웃 크기 클 경우 : 취향 비슷한 사람이 아닐 수도 있음 -> best-seller와 같음
 - 이웃 크기 작을 경우 : 너무 소수의 인원과 비슷해져 예측치 신뢰가 어려움 -> 과적합

In [11]:
rating_matrix = x_train.pivot(index='user_id',columns='movie_id',values='rating')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)
for neighbor in [10,20,30,40,50,60]:
    print(f'Neighbor size = {neighbor} : RMSE = {score_neighbor(cf_knn,x_test,neighbor)}')

Neighbor size = 10 : RMSE = 1.028178314439706
Neighbor size = 20 : RMSE = 1.0112656326880367
Neighbor size = 30 : RMSE = 1.009674537712104
Neighbor size = 40 : RMSE = 1.008874356664259
Neighbor size = 50 : RMSE = 1.0093910216446835
Neighbor size = 60 : RMSE = 1.009734588982924


## 3.6 사용자의 평가경향을 고려한 CF
 - 사용자의 평가경향을 고려해서 예측치 조정
 - 알고리즘
   1. 각 사용자의 평점평균 구함
   2. 아이템의 평점을 평균에서의 차이로 변경
   3. 평점편차를 사용해서 편차 예측값 구함, 이 아이템에 대한 평점편차와 유사도를 가중평균


In [8]:
# user별 평균을 구하고 차이를 규하여 평점편차를 구함
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

# 평점편차를 사용하여 prediction 값을 구하는 함수
def CF_knn_bias(user_id, movie_id, neighbor_size = 0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_bias[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
        
        if neighbor_size == 0:
            prediction = np.dot(sim_scores,movie_ratings)/sim_scores.sum()
            prediction += rating_mean[user_id]
        
        else:
            if len(sim_scores)>1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores,movie_ratings)/sim_scores.sum()
                prediction += rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

score_neighbor(CF_knn_bias, x_test, 30)

0.8372327629818602

In [25]:
## 사용자 ID를 지정해서 사용자를 위한 5개의 영화 추천

## 3.7 다른 CF 정확도 개선 방법
 - 신뢰도 가중치를 주는 방법(살짝 이해가 안감)
   - 유사도의 경우, 평가 아이템에 따라 신뢰도가 달라짐
   - 공통 아이템이 많은 경우, 더 큰 가중치를 주는 방법 -> but, 공통 아이템의 수를 직접 사용할 경우, RMSE가 크게 변경될 것임
   - 신뢰도가 일정 이상인 사용자만을 예측치 계산에 이용함 -> 이전에 사용하는 유사 고객이 1명 이상인 경우 사용하는 것을 일반화하여 사용

In [13]:
rating_binary1 = np.array((rating_matrix>0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index = rating_matrix.index, columns = rating_matrix.index).fillna(0)

def CF_knn_bias_sig(user_id, movie_id, neighbor_size = 0):
    if movie_id in rating_bias:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_bias[movie_id].copy()
        no_rating = movie_ratings.isnull()
        common_counts = counts[user_id]
        low_significance = common_counts < SIG_LEVEL
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        sim_scores = sim_scores.drop(none_rating_idx)
        movie_ratings = movie_ratings.drop(none_rating_idx)
        
        if neighbor_size == 0 :
            prediction = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
            prediction += rating_mean[user_id]
        
        else:
            if len(sim_scores) > MIN_RATINGS:
                neighbor_size = min(len(sim_scores), neighbor_size)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                prediction = np.dot(sim_scores, movie_ratings)/sim_scores.sum()
                prediction += rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    
    if prediction >=5 :
        prediction = 5
    elif prediction < 1 :
        prediction = 1
    return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
score_neighbor(CF_knn_bias_sig,x_test,30)                

0.8379572957500093

## 3.8 사용자 기반 CF와 아이템 기반 CF
 - UBCF : 사용자 기반 CF - 취향이 비슷한 이웃 사용자를 알아내고, 공통적으로 좋게 평가한 아이템을 추천
   - 사용자별 맞춤 추천 -> 데이터가 풍부한 경우, 정확한 추천 가능
   - 터무니 없는 추천 가능성 존재, 데이터를 자주 업데이트 해야함
   - 데이터 크기가 적고 사용자에 대한 충분한 정보 있을 경우 사용
 - IBCF : 아이템 기반 CF - 아이템 간의 유사도를 바탕으로 특정 아이템에 대한 예측 평점을 계산
   - 사용자별로 따로 계산이 들어가지 않기 때문에, 계산이 빠르다는 장점 존재
   - 데이터 업데이트가 잘 필요하지 않음
   - 데이터가 크거나 사용자에 대한 충분한 정보 없는 경우 사용
   - 아마존 같이 대용량 데이터 처리하는 곳에서는 IBCF 사용
 - 이 둘의 차이는 유사도를 계산하는 기준이 사용자인가 혹은 아이템인가의 차이

In [15]:
# 아이템 cosine 유사도 계산
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index = rating_matrix_t.index, columns = rating_matrix_t.index)

# 아이템 기반 rating 계산하는 함수
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:
        sim_scores = item_similarity[movie_id]
        user_rating = rating_matrix_t[user_id]
        non_rating_idx = user_rating[user_rating.isnull()].index
        user_rating = user_rating.dropna()
        sim_scores = sim_scores.drop(non_rating_idx)
        mean_rating = np.dot(sim_scores, user_rating)/sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

score(CF_IBCF, x_test)

0.9773086405135203

## 3.9 추천 시스템의 성과측정지표
 - 정확도는 예측값과 실제값의 차이
   - MAD
   - MSE
   - RMSE
 - 데이터가 연속값이 아닐 경우
   - accuracy, recall, precision, F1-score